# NTRU Prime 
segundo o paper **NTRU Prime** de *D.J. Bernstein, C. Chuengsatiansup, T. Lange1 and C. van Vredendaal*.

**Parâmetros e geração dos anéis de polinómios**

In [2]:
t = 64
q = 24*t
while True:
    if (1+q).is_prime():
        break
    else:
        q += 3
q += 1

Zx.<x>  = ZZ[]
Z3.<y>  = PolynomialRing(GF(3))
Gq.<z>  = GF(q)[]

p = next_prime(2*t)
while True:
    if  Gq(x^p-x-1).is_irreducible():
        break
    else:
        p = next_prime(p+1)

Zxr.<x> = Zx.quotient(x^p-x-1)
Z3r.<y> = Z3.quotient(y^p-y-1)
Gqr.<z> = Gq.quotient(z^p-z-1)

# lifting

def _lift(w):
    return Zx(map(lift,w.list()))

In [3]:
# Teste de "casting"
# i.e. converter um polinómio do anel mais geral para um anel mais específico
u = Zx([1,0,0,1,3400000003,10000000031]) ; a = Z3r(u) ; b = Gqr(u)
print a ; print b

2*y^5 + y^4 + y^3 + 1
648*z^5 + 1046*z^4 + z^3 + 1


###  Funções auxiliares

In [5]:
# random polynomial: generadores, o operador weight e o "arredondamento"

from random import choice, randint

def _small_poly(p,t=None):
    if not t:
        return Zx([choice([-1,0,1]) for k in range(p)])
    u = floor(2*(p-1)//t) ; k = randint(0,u) ; l = [0]*p
    while k < p:
        l[k] = choice([-1,1]) ; k += randint(1,u)
    return Zx(l)

def _is_small(w):
    return reduce(lambda x,y : x and y^2 <= 1,w.list(),True)

def _weight(w):
    return reduce(lambda x,y: x+1 if y!=0 else x,w.list()) 

def _round(w,n=q):
    """
         input:  polinómio em Gqr ou Z3r
         output: transpõe os coeficientes para o intervalo -n//2..+n//2
    """
    r = n//2
    return Zx(map(lambda x: lift(x + r) - r , w.list()))

def _round_3(w):
    """
         transpõe os coeficientes de "w" para o intervalo -q//2..+q//2
         e arredonda-os ao múltiplo de 3 mais próximo
    """
    def _f(x):
        return ((x/3).round())*3
    r = q//2
    return  Zx(map(lambda x: _f(lift(x+r) - r) , w.list()))

import hashlib

def Hash(w):
    ww = reduce(lambda x,y: x + y.binary(), w.list() , "")
    return hashlib.sha256(ww).hexdigest()

**Teste e verificação**

In [7]:
#u = Gqr(_small_poly(p)) * Gqr(_small_poly(p)) 
#_round_3(u)

## A classe NTRUprime

Implementada como um KEM

In [9]:
class NTRUprime(object):
    def __init__(self):
        g = _small_poly(p)
        while not Z3r(g).is_unit():
            g = _small_poly(p)
        f = _small_poly(p,t) ; g_inv = Z3r(g)^(-1)
        self.secret = (f , g_inv)                      # chave privada é um par (Zx, Z3r)
        self.pk = Gqr(g)/Gqr(3*f)                      # chave pública em Gqr

    def encapsulate(self):
        w = _small_poly(p,t)
        key = Hash(w)
        C   = _round_3(Gqr(w)*self.pk)
        return (key, C)

    def reveal(self,C):
        (f , s) = self.secret
        e = s * Z3r(_round(Gqr(3*f) * Gqr(C))) ; w = _round(e,n=3) ;
        key = Hash(w)
        return key

**Teste e Verificação**

In [10]:
# instância e geração de chaves
K = NTRUprime()
# Cifrar
(key,C) = K.encapsulate() 
# Decifrar e verificar
key == K.reveal(C)

True